In [1]:
#FARE
from numpy import *
from scipy import *
import scipy
from scipy.sparse import *
from scipy.spatial.distance import *
from sklearn.metrics.pairwise import *

In [43]:
a=array([0,0,0,1,1,0,1,1,1,1,1])
b=array([1,1,1,1,1,0,0,0,0,0,1])
cosine(a,b)

0.53708995011372429

In [48]:
type(a)

numpy.ndarray

In [4]:
tracks_with_tags_ordered=load_npz("tracks_with_tags_bincount_ordered.npz")
tracks_with_tags_ordered

<100000x276615 sparse matrix of type '<class 'numpy.float64'>'
	with 487328 stored elements in Compressed Sparse Column format>

In [3]:
IDtracksOrdered=load("ordered_tracks.npy")
IDtracksOrdered

array([ 1563309.,  1363985.,  3705881., ...,  2005630.,  4432895.,
        3626362.])

In [10]:
tracks_with_tags_ordered[1].nonzero()[1]

array([  3982,  11056, 127137, 170251, 191177], dtype=int32)

In [2]:
#FARE
def check_matrix(X, format='csc', dtype=np.float32):
    if format == 'csc' and not isinstance(X, csc_matrix):
        return X.tocsc().astype(dtype)
    elif format == 'csr' and not isinstance(X, csr_matrix):
        return X.tocsr().astype(dtype)
    elif format == 'coo' and not isinstance(X, coo_matrix):
        return X.tocoo().astype(dtype)
    elif format == 'dok' and not isinstance(X, dok_matrix):
        return X.todok().astype(dtype)
    elif format == 'bsr' and not isinstance(X, bsr_matrix):
        return X.tobsr().astype(dtype)
    elif format == 'dia' and not isinstance(X, dia_matrix):
        return X.todia().astype(dtype)
    elif format == 'lil' and not isinstance(X, lil_matrix):
        return X.tolil().astype(dtype)
    else:
        return X.astype(dtype)

In [3]:
#FARE
shrinkage=1

In [4]:
#FARE
def apply_shrinkage(X, dist):
        # create an "indicator" version of X (i.e. replace values in X with ones)
        X_ind = X.copy()
        X_ind.data = np.ones_like(X_ind.data)
        # compute the co-rated counts
        co_counts = X_ind * X_ind.T
        # remove the diagonal
        co_counts = co_counts - dia_matrix((co_counts.diagonal()[scipy.newaxis, :], [0]), shape=co_counts.shape)
        # compute the shrinkage factor as co_counts_ij / (co_counts_ij + shrinkage)
        # then multiply dist with it
        co_counts_shrink = co_counts.copy()
        co_counts_shrink.data += shrinkage
        co_counts.data /= co_counts_shrink.data
        dist.data *= co_counts.data
        return dist

In [5]:
#FARE
def myCos(X):
    X=check_matrix(X, 'csc', dtype=float32)
    Xsq=X.copy()
    Xsq.data **= 2
    norm= sqrt(Xsq.sum(axis=0))
    norm=asarray(norm).ravel()
    norm+= 1e-6
    col_nnz=diff(X.indptr)
    X.data /= repeat(norm,col_nnz)
    print("normalized")
    
    dist=X*X.T
    print("Computed")
    
    # zero out diagonal values
    dist = dist - dia_matrix((dist.diagonal()[scipy.newaxis, :], [0]), shape=dist.shape)
    print("Removed diagonal")
    if shrinkage > 0:
       dist = apply_shrinkage(X, dist)
       print("Applied shrinkage")    
        
    return dist

In [22]:
type(tracks_with_tags_ordered[0])

scipy.sparse.csc.csc_matrix

In [6]:
#FARE
playlist_with_tags=load_npz("playlists_with_tags.npz")
playlist_with_tags

<10000x276615 sparse matrix of type '<class 'numpy.float64'>'
	with 559049 stored elements in Compressed Sparse Column format>

In [7]:
#FARE
PLsimilarity=myCos(playlist_with_tags)

normalized
Computed
Removed diagonal
Applied shrinkage


In [67]:
argmax(PLsimilarity[3055])

1718

In [69]:
(PLsimilarity[1718]).getcol(3055).data

array([ 1.93072712], dtype=float32)

In [8]:
type(PLsimilarity)

scipy.sparse.csc.csc_matrix

In [73]:
save_npz("playlist_similarity.npz",PLsimilarity)